# Cuda Password Crack

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ffozre1g
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ffozre1g
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-bzylw3z1/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Kaggle". Running its setup...
Updating the package lists...
Installing nvidia-cuda-toolkit, this may take a few minutes...
Source files will be saved in "/tmp/tmp4gtan6ul".


In [ ]:
%%writefile cudacrack.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// GPU encryption function
// GPU encryption function
__device__ void CudaCrypt(char* rawPassword, char* newPassword) {
    // Encryption logic for each input character
    newPassword[0] = rawPassword[0] + 2;
    newPassword[1] = rawPassword[0] - 2;
    newPassword[2] = rawPassword[0] + 1;
    newPassword[3] = rawPassword[1] + 3;
    newPassword[4] = rawPassword[1] - 3;
    newPassword[5] = rawPassword[1] - 1;
    newPassword[6] = rawPassword[2] + 2;
    newPassword[7] = rawPassword[2] - 2;
    newPassword[8] = rawPassword[3] + 4;
    newPassword[9] = rawPassword[3] - 4;
    newPassword[10] = '\0';

    // Adjust the newPassword array to fit the specified ranges
    for (int i = 0; i < 10; i++) {
        if (i < 6) { // Ensure uppercase letter limits (A-Z)
            if (newPassword[i] > 90) { // 90 = 'Z'
                newPassword[i] = (newPassword[i] - 90) + 65; // 65 = 'A'
            } else if (newPassword[i] < 65) { // 65 = 'A'
                newPassword[i] = (65 - newPassword[i]) + 65;
            }
        } else { // Ensure numeric limits (0-9)
            if (newPassword[i] > 57) { // 57 = '9'
                newPassword[i] = (newPassword[i] - 57) + 48; // 48 = '0'
            } else if (newPassword[i] < 48) { // 48 = '0'
                newPassword[i] = (48 - newPassword[i]) + 48;
            }
        }
    }
}

// GPU kernel function for encryption
__global__ void encryptKernel(char* d_rawPassword, char* d_encryptedPassword) {
    CudaCrypt(d_rawPassword, d_encryptedPassword);
}

int main(int argc, char** argv) {
    // Ensure a password argument is provided
    if (argc != 2 || strlen(argv[1]) != 4) {
        fprintf(stderr, "Usage: %s <4-character password>\n", argv[0]);
        return 1;
    }

    // Get the password from the command-line argument
    char rawPassword[11] = {0};
    strncpy(rawPassword, argv[1], 4);
    rawPassword[4] = '\0'; // Ensure null termination

    // Allocate memory for encrypted password on the host
    char encryptedPassword[11] = {0};

    // Allocate memory on the device
    char *d_rawPassword, *d_encryptedPassword;
    cudaMalloc((void**)&d_rawPassword, sizeof(rawPassword));
    cudaMalloc((void**)&d_encryptedPassword, sizeof(encryptedPassword));

    // Copy user input to the GPU
    cudaMemcpy(d_rawPassword, rawPassword, sizeof(rawPassword), cudaMemcpyHostToDevice);

    // Launch encryption kernel
    encryptKernel<<<1, 1>>>(d_rawPassword, d_encryptedPassword);

    // Wait for the kernel to finish
    cudaDeviceSynchronize();

    // Copy the encrypted password back to the host
    cudaMemcpy(encryptedPassword, d_encryptedPassword, sizeof(encryptedPassword), cudaMemcpyDeviceToHost);

    // Display the encrypted password
    printf("Encrypted Password: %s\n", encryptedPassword);

    // Cleanup device memory
    cudaFree(d_rawPassword);
    cudaFree(d_encryptedPassword);

    return 0;
}



Overwriting cudacrack.cu


In [ ]:
%%writefile passwordCracker.cu

#include <stdio.h>
#include <string.h>

// GPU encryption function using uppercase letters and numbers
__device__ void CudaCrypt(char* rawPassword, char* newPassword) {
    // Encryption logic for each input character
    newPassword[0] = rawPassword[0] + 2;
    newPassword[1] = rawPassword[0] - 2;
    newPassword[2] = rawPassword[0] + 1;
    newPassword[3] = rawPassword[1] + 3;
    newPassword[4] = rawPassword[1] - 3;
    newPassword[5] = rawPassword[1] - 1;
    newPassword[6] = rawPassword[2] + 2;
    newPassword[7] = rawPassword[2] - 2;
    newPassword[8] = rawPassword[3] + 4;
    newPassword[9] = rawPassword[3] - 4;
    newPassword[10] = '\0';

    // Ensure uppercase letter limits (A-Z)
    for (int i = 0; i < 6; i++) {
        if (newPassword[i] > 90) { // 'Z'
            newPassword[i] = (newPassword[i] - 90) + 65; // 'A'
        } else if (newPassword[i] < 65) { // 'A'
            newPassword[i] = (65 - newPassword[i]) + 65;
        }
    }

    // Ensure numeric limits (0-9)
    for (int i = 6; i < 10; i++) {
        if (newPassword[i] > 57) { // '9'
            newPassword[i] = (newPassword[i] - 57) + 48; // '0'
        } else if (newPassword[i] < 48) { // '0'
            newPassword[i] = (48 - newPassword[i]) + 48;
        }
    }
}

// Kernel function for cracking passwords
__global__ void crackKernel(char* d_target, char* d_decryptedPassword, bool* d_found) {
    // Define character sets for the password pattern 'UpperCaseLetterUpperCaseLetterNumberNumber'
    const char uppercase[] = "ABCDEFGHIJKLMNOPQRSTUVWXYZ";
    const char digits[] = "0123456789";
    const int letterSize = sizeof(uppercase) - 1;
    const int digitSize = sizeof(digits) - 1;

    // Calculate indices using thread and block indices
    int idx1 = blockIdx.x * blockDim.x + threadIdx.x; // First uppercase letter index
    int idx2 = blockIdx.y * blockDim.y + threadIdx.y; // Second uppercase letter index
    int idx3 = blockIdx.z * blockDim.z + threadIdx.z; // First digit index

    // Ensure indices are within range and password has not yet been found
    if (idx1 < letterSize && idx2 < letterSize && idx3 < digitSize && !(*d_found)) {
        // Iterate over all possible second digits
        for (int idx4 = 0; idx4 < digitSize; idx4++) {
            char rawPassword[5] = {uppercase[idx1], uppercase[idx2], digits[idx3], digits[idx4], '\0'};
            char encryptedGuess[11] = {0};

            // Encrypt the guess
            CudaCrypt(rawPassword, encryptedGuess);

            // Compare with the target
            bool match = true;
            for (int i = 0; i < 10; i++) {
                if (encryptedGuess[i] != d_target[i]) {
                    match = false;
                    break;
                }
            }

            // If found, copy the result and mark as found
            if (match) {
                for (int i = 0; i < 4; i++) {
                    d_decryptedPassword[i] = rawPassword[i];
                }
                d_decryptedPassword[4] = '\0';
                *d_found = true;
                break;
            }
        }
    }
}

int main(int argc, char** argv) {
    // Ensure the user provides a 10-character encrypted password
    if (argc != 2 || strlen(argv[1]) != 10) {
        fprintf(stderr, "Usage: %s <10-character encrypted password>\n", argv[0]);
        return 1;
    }

    // Copy the target encrypted password
    char target[11] = {0};
    strncpy(target, argv[1], 10);

    // Allocate device memory
    char* d_target;
    char* d_decryptedPassword;
    bool* d_found;
    bool h_found = false;
    cudaMalloc((void**)&d_target, sizeof(target));
    cudaMalloc((void**)&d_decryptedPassword, 5);
    cudaMalloc((void**)&d_found, sizeof(bool));

    // Copy the target to the device
    cudaMemcpy(d_target, target, sizeof(target), cudaMemcpyHostToDevice);
    cudaMemcpy(d_found, &h_found, sizeof(bool), cudaMemcpyHostToDevice);

    // Configure blocks and threads
    dim3 threadsPerBlock(6, 6, 6);
    dim3 numBlocks(5, 5, 2);

    // Launch the kernel
    crackKernel<<<numBlocks, threadsPerBlock>>>(d_target, d_decryptedPassword, d_found);
    cudaDeviceSynchronize();

    // Copy results back
    cudaMemcpy(&h_found, d_found, sizeof(bool), cudaMemcpyDeviceToHost);
    char decryptedPassword[5] = {0};
    if (h_found) {
        cudaMemcpy(decryptedPassword, d_decryptedPassword, 5, cudaMemcpyDeviceToHost);
        printf("Decrypted Password: %s\n", decryptedPassword);
    } else {
        printf("Password not found.\n");
    }

    // Cleanup device memory
    cudaFree(d_target);
    cudaFree(d_decryptedPassword);
    cudaFree(d_found);

    return 0;
}



Overwriting passwordCracker.cu


In [ ]:
!nvcc -o cudacrack cudacrack.cu

In [ ]:
!./cudacrack 'AB12'

Encrypted Password: CCBECA3162


In [ ]:
'$6$AS$EquwSMfZH6UigdniioE8VWG9qfQ/iburie8TclTB4HCYRomJtmDsM31EqQEbs5Zk09UzWMOtHoXFFmdKRKVoy/'

In [ ]:
!nvcc -o passwordCracker passwordCracker.cu


In [ ]:
!./passwordCracker 'CCBECA3162'

Decrypted Password: AB12
